## Data enrichment

### Introduction

We define enrichment as the process of augmenting your data with new variables by means of a spatial join between your data and a `Dataset` aggregated at a given spatial resolution in the CARTO Data Observatory, or in other words:

"*Enrichment is the process of adding variables to a geometry, which we call the target, (point, line, polygon…) from a spatial (polygon) dataset, which we call the source*"

We recommend you check out the [CARTOframes quickstart](/developers/cartoframes/guides/Quickstart/) since this guide uses some of the generated DataFrames as well as the [Data Discovery guide](/developers/cartoframes/guides/Data-discovery) to learn about exploring the Data Observatory catalog to find variables of interest for your analyses.

### Choose variables to enrich from the Data Observatory catalog

Let's follow up with the [Data Discovery guide](/developers/cartoframes/guides/Data-discovery), where we subscribed to the AGS demographics dataset and listed the variables available to enrich our own data.

In [1]:
from cartoframes.auth import set_default_credentials

set_default_credentials('creds.json')

In [2]:
from cartoframes.data.observatory import Catalog, Dataset, Variable, Geography
Catalog().subscriptions().datasets

[<Dataset.get('ags_businesscou_df363a87')>,
 <Dataset.get('ags_crimerisk_e9cfa4d4')>,
 <Dataset.get('ags_retailpoten_aaf25a8c')>,
 <Dataset.get('ags_sociodemogr_f510a947')>]

In [3]:
dataset = Dataset.get('ags_sociodemogr_f510a947')
variables = dataset.variables
variables

[<Variable.get('BLOCKGROUP_108673f9')> #'Geographic Identifier',
 <Variable.get('POPCY_5e23b8f4')> #'Population (2019A)',
 <Variable.get('POPCYGRP_55c4a2e5')> #'Population in Group Quarters (2019A)',
 <Variable.get('POPCYGRPI_d37c4ec')> #'Institutional Group Quarters Population (2019A)',
 <Variable.get('AGECY0004_a67ddb4f')> #'Population age 0-4 (2019A)',
 <Variable.get('AGECY0509_de076519')> #'Population age 5-9 (2019A)',
 <Variable.get('AGECY1014_7da8d6b')> #'Population age 10-14 (2019A)',
 <Variable.get('AGECY1519_7fa0333d')> #'Population age 15-19 (2019A)',
 <Variable.get('AGECY2024_3e427146')> #'Population age 20-24 (2019A)',
 <Variable.get('AGECY2529_4638cf10')> #'Population age 25-29 (2019A)',
 <Variable.get('AGECY3034_9fe52762')> #'Population age 30-34 (2019A)',
 <Variable.get('AGECY3539_e79f9934')> #'Population age 35-39 (2019A)',
 <Variable.get('AGECY4044_4d73891c')> #'Population age 40-44 (2019A)',
 <Variable.get('AGECY4549_3509374a')> #'Population age 45-49 (2019A)',
 <Vari

As we saw in the Data Discovery guide, the `ags_sociodemogr_f510a947` dataset contains socio-demographic variables aggregated to the Census block group level. 

Let's try and find a variable for total population:

In [4]:
vdf = variables.to_dataframe()
vdf[vdf['name'].str.contains('pop', case=False, na=False)]

,slug,name,description,db_type,agg_method,column_name,variable_group_id,dataset_id,id
1,POPCY_5e23b8f4,Total Population,Population (2019A),INTEGER,SUM,POPCY,None,carto-do.ags.demographics_sociodemographics_us...,carto-do.ags.demographics_sociodemographics_us...
2,POPCYGRP_55c4a2e5,POPCYGRP,Population in Group Quarters (2019A),INTEGER,SUM,POPCYGRP,None,carto-do.ags.demographics_sociodemographics_us...,carto-do.ags.demographics_sociodemographics_us...
3,POPCYGRPI_d37c4ec,POPCYGRPI,Institutional Group Quarters Population (2019A),INTEGER,SUM,POPCYGRPI,None,carto-do.ags.demographics_sociodemographics_us...,carto-do.ags.demographics_sociodemographics_us...
84,LBFCYPOP16_55556a30,LBFCYPOP16,Population Age 16+ (2019A),INTEGER,SUM,LBFCYPOP16,carto-do.ags.demographics_sociodemographics_us...,carto-do.ags.demographics_sociodemographics_us...,carto-do.ags.demographics_sociodemographics_us...
101,POPPY_3fccf966,Total population,Population (2024A),FLOAT,SUM,POPPY,None,carto-do.ags.demographics_sociodemographics_us...,carto-do.ags.demographics_sociodemographics_us...


We can store the variable instance we need by searching the Catalog by its `slug`, in this case `POPCY_5e23b8f4`:

In [5]:
variable = Variable.get('POPCY_5e23b8f4')
variable.to_dict()

{'slug': 'POPCY_5e23b8f4',
 'name': 'Total Population',
 'description': 'Population (2019A)',
 'db_type': 'INTEGER',
 'agg_method': 'SUM',
 'column_name': 'POPCY',
 'variable_group_id': None,
 'dataset_id': 'carto-do.ags.demographics_sociodemographics_usa_blockgroup_2015_yearly_2019',
 'id': 'carto-do.ags.demographics_sociodemographics_usa_blockgroup_2015_yearly_2019.POPCY'}

The `POPCY` variable contains the `SUM` of the population for blockgroup for the year 2019. Let's enrich our stores DataFrame with that variable.

### Enrich a points DataFrame

In the [CARTOframes Quickstart](/developers/cartoframes/guides/Quickstart/) you learned how to load your own data (in this case Starbucks stores) and geocode the addresses to coordinates for further analysis.

Let's start by loading those geocoded Starbucks stores:

In [6]:
from geopandas import read_file
stores_gdf = read_file('http://libs.cartocdn.com/cartoframes/files/starbucks_brooklyn_geocoded.geojson')
stores_gdf.head(5)

,cartodb_id,field_1,name,address,revenue,geometry
0,1,0,Franklin Ave & Eastern Pkwy,"341 Eastern Pkwy,Brooklyn, NY 11238",1321040.772,POINT (-73.95901 40.67109)
1,2,1,607 Brighton Beach Ave,"607 Brighton Beach Avenue,Brooklyn, NY 11235",1268080.418,POINT (-73.96122 40.57796)
2,3,2,65th St & 18th Ave,"6423 18th Avenue,Brooklyn, NY 11204",1248133.699,POINT (-73.98976 40.61912)
3,4,3,Bay Ridge Pkwy & 3rd Ave,"7419 3rd Avenue,Brooklyn, NY 11209",1185702.676,POINT (-74.02744 40.63152)
4,5,4,Caesar's Bay Shopping Center,"8973 Bay Parkway,Brooklyn, NY 11214",1148427.411,POINT (-74.00098 40.59321)


**Note: Alternatively, you can load data in any geospatial format supported by GeoPandas or CARTO.**

As we can see, for each store we have its name, address, the total revenue by year and a `geometry` column indicating the location of the store. This is important because for the enrichment service to work, we need a DataFrame with a geometry column encoded as a [shapely](https://pypi.org/project/Shapely/) object.

We can now create a new `Enrichment` instance, and since the `stores_gdf` dataset represents store locations (points), we can use the `enrich_points` function passing as arguments, the stores DataFrame and a list of `Variables` (that we have a valid subscription from the Data Observatory catalog for).

In this case we are only enriching one variable (the total population), but we could enrich a list of them.

In [7]:
from cartoframes.data.observatory import Enrichment
enriched_stores_gdf = Enrichment().enrich_points(stores_gdf, [variable])
enriched_stores_gdf.head(5)

,cartodb_id,field_1,name,address,revenue,geometry,POPCY,do_area
0,1,0,Franklin Ave & Eastern Pkwy,"341 Eastern Pkwy,Brooklyn, NY 11238",1321040.772,POINT (-73.95901 40.67109),2215,59840.196748
1,2,1,607 Brighton Beach Ave,"607 Brighton Beach Avenue,Brooklyn, NY 11235",1268080.418,POINT (-73.96122 40.57796),1831,60150.637177
2,3,2,65th St & 18th Ave,"6423 18th Avenue,Brooklyn, NY 11204",1248133.699,POINT (-73.98976 40.61912),745,38950.618828
3,4,3,Bay Ridge Pkwy & 3rd Ave,"7419 3rd Avenue,Brooklyn, NY 11209",1185702.676,POINT (-74.02744 40.63152),1174,57353.293114
4,5,4,Caesar's Bay Shopping Center,"8973 Bay Parkway,Brooklyn, NY 11214",1148427.411,POINT (-74.00098 40.59321),2289,188379.242624


Once the enrichment finishes, there is a new column in our DataFrame called `POPCY` with population projected for the year 2019, from the US Census block group which contains each one of our Starbucks stores.

All this information, is available in the `ags_sociodemogr_e92b1637` metadata. Let's take a look:

In [8]:
dataset.to_dict()

{'slug': 'ags_sociodemogr_f510a947',
 'name': 'Sociodemographics - United States of America (Census Block Group)',
 'description': 'Census and ACS sociodemographic data estimated for the current year and data projected to five years. Projected fields are general aggregates (total population, total households, median age, avg income etc.)',
 'category_id': 'demographics',
 'country_id': 'usa',
 'data_source_id': 'sociodemographics',
 'provider_id': 'ags',
 'geography_name': 'Census Block Group - United States of America',
 'geography_description': 'Shoreline clipped TIGER/Line boundaries. More info: https://carto.com/blog/tiger-shoreline-clip/',
 'temporal_aggregation': 'yearly',
 'time_coverage': '[2019-01-01, 2020-01-01)',
 'update_frequency': 'yearly',
 'is_public_data': False,
 'lang': 'eng',
 'version': '2019',
 'category_name': 'Demographics',
 'provider_name': 'Applied Geographic Solutions',
 'geography_id': 'carto-do-public-data.carto.geography_usa_blockgroup_2015',
 'id': 'cart

### Enrich a polygon DataFrame

Next, let's do a second enrichment, but this time using a DataFrame with areas of influence calculated using the [CARTOframes isochrones](/developers/cartoframes/reference/#heading-Isolines) service to obtain the polygon around each store that covers the area within an 8, 17 and 25 minute walk.

In [9]:
aoi_gdf = read_file('http://libs.cartocdn.com/cartoframes/files/starbucks_brooklyn_isolines.geojson')
aoi_gdf.head(5)

,data_range,lower_data_range,range_label,geometry
0,500,0,8 min.,"MULTIPOLYGON (((-73.95959 40.67571, -73.95971 ..."
1,1000,500,17 min.,"POLYGON ((-73.95988 40.68110, -73.95863 40.681..."
2,1500,1000,25 min.,"POLYGON ((-73.95986 40.68815, -73.95711 40.688..."
3,500,0,8 min.,"MULTIPOLYGON (((-73.96185 40.58321, -73.96231 ..."
4,1000,500,17 min.,"MULTIPOLYGON (((-73.96684 40.57483, -73.96830 ..."


In this case we have a DataFrame which, for each index in the `stores_gdf`, contains a polygon of the areas of influence around each store at 8, 17 and 25 minute walking intervals. Again the `geometry` is encoded as a `shapely` object.

In this case, the `Enrichment` service provides an `enrich_polygons` function, which in its basic version, works in the same way as the `enrich_points` function. It just needs a DataFrame with polygon geometries and a list of variables to enrich:

In [10]:
from cartoframes.data.observatory import Enrichment
enriched_aoi_gdf = Enrichment().enrich_polygons(aoi_gdf, [variable])
enriched_aoi_gdf.head(5)

,data_range,lower_data_range,range_label,geometry,POPCY
0,500,0,8 min.,"MULTIPOLYGON (((-73.95959 40.67571, -73.95971 ...",21112.458335
1,1000,500,17 min.,"POLYGON ((-73.95988 40.68110, -73.95863 40.681...",60157.083956
2,1500,1000,25 min.,"POLYGON ((-73.95986 40.68815, -73.95711 40.688...",110657.471670
3,500,0,8 min.,"MULTIPOLYGON (((-73.96185 40.58321, -73.96231 ...",23505.104589
4,1000,500,17 min.,"MULTIPOLYGON (((-73.96684 40.57483, -73.96830 ...",29781.046917


We now have a new column in our areas of influence DataFrame, `SUM_POPCY` which represents the `SUM` of total population in the Census block groups that instersect with each polygon in our DataFrame.

### How enrichment works

Let's take a deeper look into what happens under the hood when you execute a polygon enrichment.

Imagine we have polygons representing municipalities, in blue, each of which have a population attribute, and we want to find out the population inside the green circle.

![Enrichment Schema](img/enrichment/enrichment_01.png)

We don’t know how the population is distributed inside these municipalities. They are probably concentrated in cities somewhere, but, since we don’t know where they are, our best guess is to assume that the population is evenly distributed in the municipality (i.e. every point inside the municipality has the same population density).

Population is an extensive property (it grows with area), so we can subset it (a region inside the municipality will always have a smaller population than the whole municipality), and also aggregate it by summing.

In this case, we’d calculate the population inside each part of the circle that intersects with a municipality.

**Default aggregation methods**

In the Data Observatory, we suggest a default aggregation method for certain fields. However, some fields don’t have a clear best method, and some just can’t be aggregated. In these cases, we leave the `agg_method` field blank and let the user choose the method that best fits their needs.

### Conclusion

In this guide you've seen how to use CARTOframes in conjunction with the Data Observatory to enrich a Starbucks dataset with a new population variable for the use case of revenue prediction analysis by:

- Choosing the total population variable from the Data Observatory catalog
- Calculating the sum of total population for each store
- Calculating the sum of total population around the walking areas of influence around each store

In addition, you were introduced to some more advanced concepts and further explanation of how the enrichment itself works.